In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from gensim.models import Word2Vec, FastText

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/DATASET/twetter.csv') 

# Convert labels to the appropriate data type
data['label'] = data['label'].astype(int)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.2, random_state=42)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure consistent length
max_sequence_length = max([len(sequence) for sequence in X_train_sequences])
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

# Word Embedding - Word2Vec
word2vec_model = Word2Vec(sentences=X_train_sequences, vector_size=100, min_count=1)
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv.key_to_index:
        embedding_matrix[i] = word2vec_model.wv[word]

# Word Embedding - FastText
fasttext_model = FastText(sentences=X_train_sequences, vector_size=100, min_count=1)
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
for word, i in tokenizer.word_index.items():
    if word in fasttext_model.wv.key_to_index:
        embedding_matrix[i] = fasttext_model.wv[word]

# Define the BI-LSTM model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train.astype(float), validation_data=(X_test_padded, y_test.astype(float)), epochs=10, batch_size=64)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test.astype(float), verbose=0)
print("Accuracy:", accuracy)

Epoch 1/10
400/400 [==============================] - 47s 110ms/step - loss: 0.2691 - accuracy: 0.9301 - val_loss: 0.2572 - val_accuracy: 0.9287
Epoch 2/10
400/400 [==============================] - 41s 103ms/step - loss: 0.2537 - accuracy: 0.9301 - val_loss: 0.2579 - val_accuracy: 0.9287
Epoch 3/10
400/400 [==============================] - 42s 106ms/step - loss: 0.2543 - accuracy: 0.9301 - val_loss: 0.2576 - val_accuracy: 0.9287
Epoch 4/10
400/400 [==============================] - 43s 107ms/step - loss: 0.2538 - accuracy: 0.9301 - val_loss: 0.2585 - val_accuracy: 0.9287
Epoch 5/10
400/400 [==============================] - 41s 104ms/step - loss: 0.2537 - accuracy: 0.9301 - val_loss: 0.2571 - val_accuracy: 0.9287
Epoch 6/10
400/400 [==============================] - 42s 105ms/step - loss: 0.2537 - accuracy: 0.9301 - val_loss: 0.2587 - val_accuracy: 0.9287
Epoch 7/10
400/400 [==============================] - 41s 103ms/step - loss: 0.2536 - accuracy: 0.9301 - val_loss: 0.2572 - val_ac